In [1]:
import pandas as pd
import numpy as np

In [2]:
# 데이터 로드
df_train = pd.read_csv('train.csv')
star_train = pd.read_csv('starstar.csv')

In [3]:
df_train.head()

,game_id,winner,time,player,species,event,event_contents
0,0,1,0.00,0,T,Camera,"at (145.25, 21.5078125)"
1,0,1,0.00,1,T,Camera,"at (22.75, 147.0078125)"
2,0,1,0.02,0,T,Selection,['OrbitalCommand [3080001]']
3,0,1,0.02,0,T,Ability,(1360) - TrainSCV
4,0,1,0.14,0,T,Camera,"at (142.99609375, 24.50390625)"


In [4]:
star_train.head()

,game_id,time,player0_species,player1_species,event_count_0,event_count_1,Camera_count_0,Camera_count_1,Selection_count_0,Selection_count_1,...,ControlGroup_count_1,Camera_diff,Selection_diff,Ability_diff,Right Click_diff,SetControlGroup_diff,GetControlGroup_diff,AddToControlGroup_diff,ControlGroup_diff,underTime9
0,0,7.24,0,0,592,548,444.0,425.0,50.0,57.0,...,0.0,19.0,-7.0,0.0,7.0,2.0,21.0,2.0,0.0,0
1,1,9.59,1,0,1223,1211,627.0,858.0,186.0,116.0,...,0.0,-231.0,70.0,10.0,29.0,2.0,131.0,1.0,0.0,1
2,2,9.59,1,2,851,1371,413.0,725.0,90.0,232.0,...,2.0,-312.0,-142.0,-16.0,-44.0,5.0,-10.0,1.0,-2.0,1
3,3,9.59,0,1,1389,1622,713.0,388.0,180.0,148.0,...,0.0,325.0,32.0,-7.0,8.0,-13.0,-578.0,0.0,0.0,1
4,4,5.43,0,2,966,651,430.0,272.0,67.0,126.0,...,0.0,158.0,-59.0,21.0,71.0,2.0,125.0,-3.0,0.0,0


# 추가 할 columns  
1. player별 event_contents 카운트  
    1) 각 event별로 event_contents의 내용을 전처리 하여 카테고리화 시킨다.  
    2) 0 player는 event_contents_0으로, 1 player는 event_contents_1로 피처명을 정하고 각 게임별로 카운트한다.  
2. plyaer 간의 event_contents의 차이를 나타내는 diff columns  
    1) event_contetns_diff = 0 player - 1 player

In [5]:
star_train

,game_id,time,player0_species,player1_species,event_count_0,event_count_1,Camera_count_0,Camera_count_1,Selection_count_0,Selection_count_1,...,ControlGroup_count_1,Camera_diff,Selection_diff,Ability_diff,Right Click_diff,SetControlGroup_diff,GetControlGroup_diff,AddToControlGroup_diff,ControlGroup_diff,underTime9
0,0,7.24,0,0,592,548,444.0,425.0,50.0,57.0,...,0.0,19.0,-7.0,0.0,7.0,2.0,21.0,2.0,0.0,0
1,1,9.59,1,0,1223,1211,627.0,858.0,186.0,116.0,...,0.0,-231.0,70.0,10.0,29.0,2.0,131.0,1.0,0.0,1
2,2,9.59,1,2,851,1371,413.0,725.0,90.0,232.0,...,2.0,-312.0,-142.0,-16.0,-44.0,5.0,-10.0,1.0,-2.0,1
3,3,9.59,0,1,1389,1622,713.0,388.0,180.0,148.0,...,0.0,325.0,32.0,-7.0,8.0,-13.0,-578.0,0.0,0.0,1
4,4,5.43,0,2,966,651,430.0,272.0,67.0,126.0,...,0.0,158.0,-59.0,21.0,71.0,2.0,125.0,-3.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38867,38867,8.22,0,1,831,457,639.0,324.0,53.0,58.0,...,0.0,315.0,-5.0,11.0,52.0,1.0,0.0,0.0,0.0,0
38868,38868,9.59,0,1,806,776,558.0,633.0,62.0,48.0,...,0.0,-75.0,14.0,35.0,67.0,-3.0,-8.0,0.0,0.0,1
38869,38869,8.25,1,2,1227,838,785.0,330.0,106.0,191.0,...,0.0,455.0,-85.0,3.0,81.0,-2.0,-61.0,-2.0,0.0,0
38870,38870,5.56,1,1,406,593,321.0,385.0,24.0,64.0,...,0.0,-64.0,-40.0,-16.0,-32.0,-4.0,-32.0,1.0,0.0,0


In [6]:
star_train

,game_id,time,player0_species,player1_species,event_count_0,event_count_1,Camera_count_0,Camera_count_1,Selection_count_0,Selection_count_1,...,ControlGroup_count_1,Camera_diff,Selection_diff,Ability_diff,Right Click_diff,SetControlGroup_diff,GetControlGroup_diff,AddToControlGroup_diff,ControlGroup_diff,underTime9
0,0,7.24,0,0,592,548,444.0,425.0,50.0,57.0,...,0.0,19.0,-7.0,0.0,7.0,2.0,21.0,2.0,0.0,0
1,1,9.59,1,0,1223,1211,627.0,858.0,186.0,116.0,...,0.0,-231.0,70.0,10.0,29.0,2.0,131.0,1.0,0.0,1
2,2,9.59,1,2,851,1371,413.0,725.0,90.0,232.0,...,2.0,-312.0,-142.0,-16.0,-44.0,5.0,-10.0,1.0,-2.0,1
3,3,9.59,0,1,1389,1622,713.0,388.0,180.0,148.0,...,0.0,325.0,32.0,-7.0,8.0,-13.0,-578.0,0.0,0.0,1
4,4,5.43,0,2,966,651,430.0,272.0,67.0,126.0,...,0.0,158.0,-59.0,21.0,71.0,2.0,125.0,-3.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38867,38867,8.22,0,1,831,457,639.0,324.0,53.0,58.0,...,0.0,315.0,-5.0,11.0,52.0,1.0,0.0,0.0,0.0,0
38868,38868,9.59,0,1,806,776,558.0,633.0,62.0,48.0,...,0.0,-75.0,14.0,35.0,67.0,-3.0,-8.0,0.0,0.0,1
38869,38869,8.25,1,2,1227,838,785.0,330.0,106.0,191.0,...,0.0,455.0,-85.0,3.0,81.0,-2.0,-61.0,-2.0,0.0,0
38870,38870,5.56,1,1,406,593,321.0,385.0,24.0,64.0,...,0.0,-64.0,-40.0,-16.0,-32.0,-4.0,-32.0,1.0,0.0,0


In [7]:
df_train.event.unique()

array(['Camera', 'Selection', 'Ability', 'Right Click', 'SetControlGroup',
       'GetControlGroup', 'AddToControlGroup', 'ControlGroup'],
      dtype=object)

In [8]:
# event가 Right Click인 event_contents의 데이터 전처리
df = df_train[df_train.event == 'Right Click']
event_Right_Click = df.event_contents

right_clicks = []
for index in range(len(event_Right_Click)):
    right_click = event_Right_Click.iloc[index].split(':')[0]
    if (right_click == "Target"):
        right_click = event_Right_Click.iloc[index].split('[')[0]
    right_clicks.append(right_click)
    
df.event_contents = right_clicks

C:\Users\eh063\Anaconda3\lib\site-packages\pandas\core\generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [9]:
df

,game_id,winner,time,player,species,event,event_contents
17,0,1,0.20,0,T,Right Click,Location
18,0,1,0.23,0,T,Right Click,Location
111,0,1,1.17,0,T,Right Click,Location
112,0,1,1.18,0,T,Right Click,Location
113,0,1,1.18,0,T,Right Click,Location
...,...,...,...,...,...,...,...
67091737,38871,0,8.48,0,Z,Right Click,Target: DestructibleSignsFunny
67091741,38871,0,8.48,0,Z,Right Click,Target: DestructibleSignsFunny
67091744,38871,0,8.48,0,Z,Right Click,Target: DestructibleSignsFunny
67091765,38871,0,8.50,0,Z,Right Click,Location


In [10]:
# player별로 event가 Right Click인 데이터를 event_contents별로 카운트.
for event_contents in df.event_contents.unique():
    # player가 0일 경우
    test = df[df.player == 0]
    test = pd.DataFrame(df.game_id.value_counts()[df.game_id.unique()]).rename({'game_id':event_contents +'_count_0'}, axis = 1)
    test['game_id']= np.array(test.index)
    star_train = pd.merge(star_train, test, on='game_id', how='left')
    
    # player가 1일 경우
    test = df[df.player == 1]
    test = pd.DataFrame(df.game_id.value_counts()[df.game_id.unique()]).rename({'game_id':event_contents +'_count_1'}, axis = 1)
    test['game_id']= np.array(test.index)
    star_train = pd.merge(star_train, test, on='game_id', how='left')
    
    # player간의 event_contetnts 카운트 차이를 보여주는 diff 피처 추가
    try:
        star_train[event_content + '_diff'] = star_train[event_content+'_count_0'] - star_train[event_content+'_count_1']
    except:
        pass

In [11]:
# event가 Selection인 event_contents의 데이터 전처리
df = df_train[df_train.event == 'Selection']
event_Selection = df.event_contents

selections = []
for index in range(len(event_Selection)):
    selection = event_Selection.iloc[index].split('[')[1]
    selections.append(selection)
df.event_contents = selections

In [12]:
# player별로 event가 Selection인 데이터를 event_contents별로 카운트.
for event_contents in df.event_contents.unique():
    # player가 0일 경우
    test = df[df.player == 0]
    test = pd.DataFrame(df.game_id.value_counts()[df.game_id.unique()]).rename({'game_id':event_contents +'_count_0'}, axis = 1)
    test['game_id']= np.array(test.index)
    star_train = pd.merge(star_train, test, on='game_id', how='left')
    
    # player가 1일 경우
    test = df[df.player == 1]
    test = pd.DataFrame(df.game_id.value_counts()[df.game_id.unique()]).rename({'game_id':event_contents +'_count_1'}, axis = 1)
    test['game_id']= np.array(test.index)
    star_train = pd.merge(star_train, test, on='game_id', how='left')
    
    # player간의 event_contetnts 카운트 차이를 보여주는 diff 피처 추가
    try:
        star_train[event_content + '_diff'] = star_train[event_content+'_count_0'] - star_train[event_content+'_count_1']
    except:
        pass

In [13]:
# event가 Ability인 event_contents의 데이터 전처리
df = df_train[df_train.event == 'Ability']
event_Ability = df.event_contents

abilities = []
for index in range(len(event_Ability)):
    ability = event_Ability.iloc[index].split(';')[0]
    try:
        ability = ability.split('-')[1]
    except:
        pass
    abilities.append(ability)
df.event_contents = abilities

In [14]:
# player별로 event가 Ability인 데이터를 event_contents별로 카운트.
for event_contents in df.event_contents.unique():
    # player가 0일 경우
    test = df[df.player == 0]
    test = pd.DataFrame(df.game_id.value_counts()[df.game_id.unique()]).rename({'game_id':event_contents +'_count_0'}, axis = 1)
    test['game_id']= np.array(test.index)
    star_train = pd.merge(star_train, test, on='game_id', how='left')
    
    # player가 1일 경우
    test = df[df.player == 1]
    test = pd.DataFrame(df.game_id.value_counts()[df.game_id.unique()]).rename({'game_id':event_contents +'_count_1'}, axis = 1)
    test['game_id']= np.array(test.index)
    star_train = pd.merge(star_train, test, on='game_id', how='left')
    
    # player간의 event_contetnts 카운트 차이를 보여주는 diff 피처 추가
    try:
        star_train[event_content + '_diff'] = star_train[event_content+'_count_0'] - star_train[event_content+'_count_1']
    except:
        pass

SetControlGroup, GetcontrolGroup, AddToControlGroup, ControlGroup은 모두 nan이므로 카운트 할 것이 없음.

In [15]:
df_train[df_train.event=='SetControlGroup'].event_contents.unique()

array([nan], dtype=object)

In [16]:
df_train[df_train.event=='GetControlGroup'].event_contents.unique()

array([nan], dtype=object)

In [17]:
df_train[df_train.event=='AddToControlGroup'].event_contents.unique()

array([nan], dtype=object)

In [18]:
df_train[df_train.event=='ControlGroup'].event_contents.unique()

array([nan], dtype=object)

# player별 event_contents에 따른 카운트 수 컬럼 추가(test데이터)  

In [19]:
# 데이터 로드
df_test = pd.read_csv('test.csv')
star_test = pd.read_csv('star_test.csv')

In [20]:
df_test

,game_id,time,player,species,event,event_contents
0,38872,0.00,0,P,Camera,"at (22.25, 81.5078125)"
1,38872,0.00,1,P,Camera,"at (120.25, 153.83984375)"
2,38872,0.01,1,P,Selection,['Nexus [3100001]']
3,38872,0.01,1,P,Ability,(15E0) - TrainProbe
4,38872,0.01,1,P,AddToControlGroup,NaN
...,...,...,...,...,...,...
28714844,55658,4.54,1,T,Right Click,"Location: (120.584228515625, 56.930419921875, ..."
28714845,55658,4.54,0,Z,Camera,"at (70.9921875, 117.65234375)"
28714846,55658,4.54,1,T,Right Click,"Location: (122.146728515625, 55.52099609375, 3..."
28714847,55658,4.54,1,T,Right Click,"Location: (123.6767578125, 54.140625, 33215)"


In [21]:
star_test

,game_id,time,player0_species,player1_species,event_count_0,event_count_1,Camera_count_0,Camera_count_1,Selection_count_0,Selection_count_1,...,UpgradeGroundWeapons3_count_0,UpgradeGroundWeapons3_count_1,EnableVolatileBurst_count_0,EnableVolatileBurst_count_1,LoadTargetOverlord_count_0,LoadTargetOverlord_count_1,DarkTemplarBlink_count_0,DarkTemplarBlink_count_1,HallucinateZealot_count_0,HallucinateZealot_count_1
0,38872,7.06,1,1,571,715,232.0,467.0,52.0,41.0,...,0,0,0,0,0,0,0,0,0,0
1,38873,6.07,2,1,421,389,200.0,146.0,114.0,19.0,...,0,0,0,0,0,0,0,0,0,0
2,38874,4.41,0,0,469,824,245.0,220.0,42.0,36.0,...,0,0,0,0,0,0,0,0,0,0
3,38875,7.58,0,1,1223,863,515.0,357.0,123.0,99.0,...,0,0,0,0,0,0,0,0,0,0
4,38876,2.50,2,2,582,213,129.0,161.0,25.0,35.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16782,55654,6.14,2,0,1318,1172,258.0,318.0,186.0,108.0,...,0,0,0,0,0,0,0,0,0,0
16783,55655,9.58,2,0,1571,1042,810.0,638.0,317.0,103.0,...,0,0,0,0,0,0,0,0,0,0
16784,55656,6.24,1,1,518,697,330.0,328.0,67.0,52.0,...,0,0,0,0,0,0,0,0,0,0
16785,55657,1.17,0,0,42,215,20.0,65.0,5.0,23.0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
# event가 Right Click인 event_contents의 데이터 전처리
df = df_test[df_test.event == 'Right Click']
event_Right_Click = df.event_contents

right_clicks = []
for index in range(len(event_Right_Click)):
    right_click = event_Right_Click.iloc[index].split(':')[0]
    if (right_click == "Target"):
        right_click = event_Right_Click.iloc[index].split('[')[0]
    right_clicks.append(right_click)
    
df.event_contents = right_clicks

In [23]:
# player별로 event가 Right Click인 데이터를 event_contents별로 카운트.
for event_contents in df.event_contents.unique():
    # player가 0일 경우
    test = df[df.player == 0]
    test = pd.DataFrame(df.game_id.value_counts()[df.game_id.unique()]).rename({'game_id':event_contents +'_count_0'}, axis = 1)
    test['game_id']= np.array(test.index)
    star_test = pd.merge(star_test, test, on='game_id', how='left')
    
    # player가 1일 경우
    test = df[df.player == 1]
    test = pd.DataFrame(df.game_id.value_counts()[df.game_id.unique()]).rename({'game_id':event_contents +'_count_1'}, axis = 1)
    test['game_id']= np.array(test.index)
    star_test = pd.merge(star_test, test, on='game_id', how='left')
    
    # player간의 event_contetnts 카운트 차이를 보여주는 diff 피처 추가
    try:
        star_test[event_content + '_diff'] = star_test[event_content+'_count_0'] - star_test[event_content+'_count_1']
    except:
        pass

In [24]:
# event가 Selection인 event_contents의 데이터 전처리
df = df_test[df_test.event == 'Selection']
event_Selection = df.event_contents

selections = []
for index in range(len(event_Selection)):
    selection = event_Selection.iloc[index].split('[')[1]
    selections.append(selection)
df.event_contents = selections

In [25]:
# player별로 event가 Selection인 데이터를 event_contents별로 카운트.
for event_contents in df.event_contents.unique():
    # player가 0일 경우
    test = df[df.player == 0]
    test = pd.DataFrame(df.game_id.value_counts()[df.game_id.unique()]).rename({'game_id':event_contents +'_count_0'}, axis = 1)
    test['game_id']= np.array(test.index)
    star_test = pd.merge(star_test, test, on='game_id', how='left')
    
    # player가 1일 경우
    test = df[df.player == 1]
    test = pd.DataFrame(df.game_id.value_counts()[df.game_id.unique()]).rename({'game_id':event_contents +'_count_1'}, axis = 1)
    test['game_id']= np.array(test.index)
    star_test = pd.merge(star_test, test, on='game_id', how='left')
    
    # player간의 event_contetnts 카운트 차이를 보여주는 diff 피처 추가
    try:
        star_test[event_content + '_diff'] = star_test[event_content+'_count_0'] - star_test[event_content+'_count_1']
    except:
        pass

In [26]:
# event가 Ability인 event_contents의 데이터 전처리
df = df_test[df_test.event == 'Ability']
event_Ability = df.event_contents

abilities = []
for index in range(len(event_Ability)):
    ability = event_Ability.iloc[index].split(';')[0]
    try:
        ability = ability.split('-')[1]
    except:
        pass
    abilities.append(ability)
df.event_contents = abilities

In [27]:
# player별로 event가 Ability인 데이터를 event_contents별로 카운트.
for event_contents in df.event_contents.unique():
    # player가 0일 경우
    test = df[df.player == 0]
    test = pd.DataFrame(df.game_id.value_counts()[df.game_id.unique()]).rename({'game_id':event_contents +'_count_0'}, axis = 1)
    test['game_id']= np.array(test.index)
    star_test = pd.merge(star_test, test, on='game_id', how='left')
    
    # player가 1일 경우
    test = df[df.player == 1]
    test = pd.DataFrame(df.game_id.value_counts()[df.game_id.unique()]).rename({'game_id':event_contents +'_count_1'}, axis = 1)
    test['game_id']= np.array(test.index)
    star_test = pd.merge(star_test, test, on='game_id', how='left')
    
    # player간의 event_contetnts 카운트 차이를 보여주는 diff 피처 추가
    try:
        star_test[event_content + '_diff'] = star_test[event_content+'_count_0'] - star_test[event_content+'_count_1']
    except:
        pass

In [28]:
star_test

,game_id,time,player0_species,player1_species,event_count_0,event_count_1,Camera_count_0,Camera_count_1,Selection_count_0,Selection_count_1,...,ResearchTerranVehicleAndShipArmorsLevel2_count_0_y,ResearchTerranVehicleAndShipArmorsLevel2_count_1_y,EvolveFlyerAttacks2_count_0_y,EvolveFlyerAttacks2_count_1_y,EvolveFlyerCarapace2_count_0_y,EvolveFlyerCarapace2_count_1_y,LurkerHoldFire_count_0_y,LurkerHoldFire_count_1_y,BuildPointDefenseDrone_count_0_y,BuildPointDefenseDrone_count_1_y
0,38872,7.06,1,1,571,715,232.0,467.0,52.0,41.0,...,69.0,69.0,69.0,69.0,69.0,69.0,69.0,69.0,69.0,69.0
1,38873,6.07,2,1,421,389,200.0,146.0,114.0,19.0,...,62.0,62.0,62.0,62.0,62.0,62.0,62.0,62.0,62.0,62.0
2,38874,4.41,0,0,469,824,245.0,220.0,42.0,36.0,...,62.0,62.0,62.0,62.0,62.0,62.0,62.0,62.0,62.0,62.0
3,38875,7.58,0,1,1223,863,515.0,357.0,123.0,99.0,...,153.0,153.0,153.0,153.0,153.0,153.0,153.0,153.0,153.0,153.0
4,38876,2.50,2,2,582,213,129.0,161.0,25.0,35.0,...,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16782,55654,6.14,2,0,1318,1172,258.0,318.0,186.0,108.0,...,90.0,90.0,90.0,90.0,90.0,90.0,90.0,90.0,90.0,90.0
16783,55655,9.58,2,0,1571,1042,810.0,638.0,317.0,103.0,...,137.0,137.0,137.0,137.0,137.0,137.0,137.0,137.0,137.0,137.0
16784,55656,6.24,1,1,518,697,330.0,328.0,67.0,52.0,...,50.0,50.0,50.0,50.0,50.0,50.0,50.0,50.0,50.0,50.0
16785,55657,1.17,0,0,42,215,20.0,65.0,5.0,23.0,...,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0


star_test 데이터셋도 마찬가지로 SetControlGroup, GetcontrolGroup, AddToControlGroup, ControlGroup은 모두 nan이므로 카운트 할 것이 없음.

In [29]:
# star_train과 star_test의 nan값 0으로
star_train = star_train.fillna(0)
star_test = star_test.fillna(0)

star_train과 star_test에 서로 가지고 있지 않은 피처들이 존재. 학습할 때 train 데이터셋과 test 데이터셋의 피처가 모두 같아야 하므로 서로 없는 피처들을 만들어주고 카운트를 0으로 지정함.

In [30]:
# 서로의 column 목록을 리스트 형태로 반환
train_columns = list(star_train.columns)
test_columns = list(star_test.columns)

In [31]:
# star_train 데이터
no_columns_train = []
for test_column in test_columns:
    if test_column not in train_columns:
        no_columns_train.append(test_column)
        
for column in no_columns_train:
    star_train[column] = 0

In [32]:
# star_test 데이터
no_columns_test = []
for train_column in train_columns:
    if train_column not in test_columns:
        no_columns_test.append(train_column)
        
for column in no_columns_test:
    star_test[column] = 0

In [33]:
star_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38872 entries, 0 to 38871
Columns: 4197 entries, game_id to  BuildPointDefenseDrone_count_1_y
dtypes: float64(1093), int64(3104)
memory usage: 1.2 GB


In [34]:
star_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16787 entries, 0 to 16786
Columns: 4197 entries, game_id to  SpawnInfestedTerran_count_1
dtypes: float64(2081), int64(2116)
memory usage: 537.7 MB


In [35]:
# csv 파일로 저장
star_train.to_csv('final_star_train.csv', index=False)
star_test.to_csv('final_star_test.csv', index=False)